# Code for Retirement Modelling

## Contents

* [1. Importing Libraries](#1.-Importing-Libraries)

* [2. Data Pre-processing](#2.-Data-Pre-processing)
> * [2.1. Data loading](#2.1.-Data-loading)
> * [2.2. Intial exploration of data](#2.2.-Intial-exploration-of-data)
> * [2.3. Data Cleaning, Wrangling, Feature Reduction & Feature Engineering](#2.3.-Data-Cleaning,-Wrangling,-Feature-Reduction-&-Feature-Engineering)
>> * [2.3.1. Person Number: Handling missing values](#2.3.1.-Person-Number:-Handling-missing-values)
>> * [2.3.2. Reviewing and transforming values in columns](#2.3.2.-Reviewing-and-transforming-values-in-columns)
>> * [2.3.3. Renaming features](#2.3.3.-Renaming-features)
>> * [2.3.4. Dropping features](#2.3.4.-Dropping-features)
>> * [2.3.5. Feature Engineering & Creation](#2.3.5.-Feature-Engineering-&-Creation)
>> * [2.3.6. Dealing with the period of the Pandemic](#2.3.6.-Dealing-with-the-period-of-the-Pandemic)
> * [2.4. Visualising features](#2.4.-Visualising-features)
>> * [2.4.1. Bar plots](#2.4.1.-Bar-plots)
>> * [2.4.2. Histograms](#2.4.2.-Histograms)
> * [2.5. Feature Relationship](#2.5.-Feature-Relationship)
>> * [2.5.1. Calculation of Bivariate Stats](#2.5.1.-Calculation-of-Bivariate-Stats)
>> * [2.5.2. Pairplots](#2.5.2.-Pairplots)
>> * [2.5.3. Heatmap](#2.5.3.-Heatmap)

* [3. Modelling Preparation](#3.-Modelling-Preparation)
> * [3.1. Final feature changes](#3.1.-Final-feature-changes)
> * [3.2. One Hot Encoding](#3.2.-One-Hot-Encoding)
> * [3.3. Normalisation](#3.3.-Normalisation)
> * [3.4. Training - Testing split](#3.4.-Training---Testing-split)

* [4. Classification Models](#4.-Classification-Models)
> * [4.1. Random Forest](#4.1.-Random-Forest)
>> * [4.1.1. Random Forest - First Run](#4.1.1.-Random-Forest---First-Run)
>> * [4.1.2. Random Forest - Tuning Hyperparameters](#4.1.2.-Random-Forest---Tuning-Hyperparameters)
> * [4.2. Support Vector Machines (SVM)](#4.2.-Support-Vector-Machines-(SVM))
>> * [4.2.1. SVM - First Run](#4.2.1.-SVM---First-Run)
>> * [4.2.2. SVM - Tuning hyperparameters](#4.2.2.-SVM---Tuning-hyperparameters)
> * [4.3. Logistic Regression](#4.3.-Logistic-Regression)
>> * [4.3.1. Logistic Regression - First Run](#4.3.1.-Logistic-Regression---First-Run)
>> * [4.3.2. Logistic Regression - Tuning hyperparameters](#4.3.2.-Logistic-Regression---Tuning-hyperparameters)
> * [4.4. K-Nearest Neighbour (KNN)](#4.4.-K-Nearest-Neighbour-(KNN))
>> * [4.4.1. KNN - First Run](#4.4.1.-KNN---First-Run)
>> * [4.4.2. KNN - Tuning hyperparameters](#4.4.2.-KNN---Tuning-hyperparameters)



## 1. Importing Libraries

In [ ]:
import datetime
import time

import pandas as pd
from scipy import stats
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import statistics

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

## 2. Data Pre-processing

### 2.1. Data loading

#### Importing the data files

This is the data that I have exported from our internal HR System.

In [ ]:
data_pension         = pd.read_csv("HMLR_121 Pension Scheme - anonymised.csv")
data_additional_info = pd.read_csv("Retirement Modelling Test report - anonymised.csv")
data_leavers         = pd.read_csv("HMLR_026 Leavers Report - April 2018 - March 2023 - anonymised.csv")

pension_shape         = str(data_pension.shape)
additional_info_shape = str(data_additional_info.shape)
leavers_shape         = str(data_leavers.shape)

print("The size of the pension data is: " + pension_shape)
print("The size of the additional information is: " + additional_info_shape)
print("The size of the leavers data is: " + leavers_shape)

The head method is used to quickly review each of the datasets.

##### Pension Scheme Data

In [ ]:
data_pension.head()

In [ ]:
data_pension.info()

##### Personal data

In [ ]:
data_additional_info.head()

In [ ]:
data_additional_info.info()

##### Leavers Data

In [ ]:
data_leavers.head()

In [ ]:
data_leavers.info()

#### Combining all imported dataframes

The code below is to merge the imported dataframes using the uinque Person Number (Employee ID).

In [ ]:
data_merge_stage_1 = pd.merge(data_leavers, data_pension, on='Person Number', how='left')
data_total_merge   = pd.merge(data_merge_stage_1, data_additional_info, on='Person Number', how='left')

# reviewing the resulting dataframe

data_total_merge.head(6)

In [ ]:
# Code to review the shape of the new dataframe

combined_shape = str(data_total_merge.shape)

print("The combined size of the data extracts is: " + combined_shape)

### 2.2. Intial exploration of data

The following function below is to assist with collecting various univariate stats and loading them into a new dataframe to assist with understanding the dataset and start the process of cleaning and transforming the data before delving into more detailed data exploration.

In [ ]:
def univariate_stats(df):
    import pandas as pd
    output_df = pd.DataFrame(columns=['Count','Missing','Unique','Dtype','Numeric','Mode','Mean','Min','25%','Median',
                                      '75%','Max','Std','Skew','Kurtosis'])
    
    for col in df:
        if pd.api.types.is_numeric_dtype(df[col]):
            output_df.loc[col] = [df[col].count(), df[col].isnull().sum(), df[col].nunique(), df[col].dtype,
                                  pd.api.types.is_numeric_dtype(df[col]), df[col].mode().values[0], df[col].mean(),
                                  df[col].min(), df[col].quantile(0.25), df[col].median(), df[col].quantile(0.75),
                                  df[col].max(), df[col].std(), df[col].skew(), df[col].kurt()]
        
        else:
            output_df.loc[col] = [df[col].count(), df[col].isnull().sum(), df[col].nunique(), df[col].dtype,
                                  pd.api.types.is_numeric_dtype(df[col]), df[col].mode().values[0], '-','-','-','-',
                                  '-','-','-','-','-']
            
    return output_df.sort_values(by=['Numeric', 'Skew', 'Unique'], ascending=False)

In [ ]:
univariate_stats(data_total_merge)

### 2.3. Data Cleaning, Wrangling, Feature Reduction & Feature Engineering

After an intial review of the univariate stats there are a number of Cleaning and Wrangling steps that are required before exploring the data in more detail. These steps include:

- Calculating Age.
- Calculating Length of Service (LOS) - both Civil Service (CS) and His Majesty's Land Registry (HMLR).
- Reviewing duplicate entries.
- Dropping any features that are not required or may add bias to the data when running through the machine learning algorithms.
- Transforming categorical features into numeric fields where they are a binary option.
- Data transformation of values in columns.


#### 2.3.1. Person Number: Handling missing values

##### Duplicate values

Using the univariate DataFrame we can see that there is a duplicate row of data. My knowledge of the organisations HR database I know that during migration there were some issues with some employee assignment records and these duplicates are expected and we can delete one of the rows as it is due to a data migration error.

In [ ]:
# Showing the duplicate exists as the Unique value is not the same as the Count value.

univariate_stats(data_total_merge).loc['Person Number']

In [ ]:
# Code to remove the second instance of the duplicated row of data.

data_total_merge = data_total_merge.drop_duplicates(subset=['Person Number'], keep='first')

# Showing the results of dropping the duplicate.

univariate_stats(data_total_merge).loc['Person Number']

The univariate statistics show that the `Usage Code for Person : HMLR and CS General Entry Dates : CS Current Entry Date` has a missing value. Given that this is a date value it is only a single missing value I have decided to drop this row of data.

In [ ]:
# Confirming that there is a missing value in this feature

univariate_stats(data_total_merge).loc['Usage Code for Person : HMLR and CS General Entry Dates : CS Current Entry Date']

In [ ]:
# Dropping the missing value

data_total_merge.dropna(subset=['Usage Code for Person : HMLR and CS General Entry Dates : CS Current Entry Date'],
                        inplace=True)

In [ ]:
# Reviewing the result of dropping the missing value

univariate_stats(data_total_merge).loc['Usage Code for Person : HMLR and CS General Entry Dates : CS Current Entry Date']

#### 2.3.2. Reviewing and transforming values in columns

`Assignment Category`

The Assignment Category field holds a number of different entries related to staff. There are two entries:
`Partial retirement` and `Partial retirement PartTime` 
that are needed and the rest are not relevant. 

I will create a field to show a binary result as to whether an employee is partially retired or not and then the assignment category field will be dropped.

In [ ]:
# Identifying the unique values in the column

print(data_total_merge['Assignment Category'].unique())

In [ ]:
# Code to transform the data into a binary output of Partially Retired or not into a new column.

data_total_merge.loc[(data_total_merge['Assignment Category'] == 'Partial retirement') | 
                     (data_total_merge['Assignment Category'] == 'Partial retirement PartTime'), 'Partial_Retirement'] = 1  

data_total_merge.loc[(data_total_merge['Assignment Category'] != 'Partial retirement') & 
                     (data_total_merge['Assignment Category'] != 'Partial retirement PartTime'), 'Partial_Retirement'] = 0

`Element Name`

The Element Name lists the Pension Scheme that employees are a part of but there are a number of different entries that are related to the Schemes. The following code will clean the data to reduce the list to the specific Pension Schemes.

In [ ]:
# Identifying the unique values in the column

print(data_total_merge['Element Name'].unique())

In [ ]:
# Replace Element Name column with Pension data as Alpha, Classic, Classic Plus and the rest of the Pension Schemes.

data_total_merge.loc[(data_total_merge['Element Name'] == 'Classic Civil Servant Pension') | (data_total_merge['Element Name'] == 'Classic CS Added'), 'Pension Scheme'] = 'Classic' 
data_total_merge.loc[(data_total_merge['Element Name'] == 'Premium Civil Servant Pension') | (data_total_merge['Element Name'] == 'Premium CS Added'), 'Pension Scheme'] = 'Premium'
data_total_merge.loc[(data_total_merge['Element Name'] == 'Alpha Pension Scheme') | (data_total_merge['Element Name'] == 'Alpha EPA 1 2015'), 'Pension Scheme'] = 'Alpha' 
data_total_merge.loc[(data_total_merge['Element Name'] == 'Add Pension Member (Alpha) 2015') | (data_total_merge['Element Name'] == 'Additional Pension (Alpha) 2015'), 'Pension Scheme'] = 'Alpha'
data_total_merge.loc[(data_total_merge['Element Name'] == 'Classic Plus Civil Servant Pension') | (data_total_merge['Element Name'] == 'Classic Plus CS Added'), 'Pension Scheme'] = 'Classic Plus'
data_total_merge.loc[(data_total_merge['Element Name'] == 'Nuvos Civil Servant Pension'), 'Pension Scheme'] = 'Nuvos' 
data_total_merge.loc[(data_total_merge['Element Name'] == 'L&G Defined Contribution Pension'), 'Pension Scheme'] = 'L&G'

From reviewing the unique value output there are rows of data that do not have a `Pension Scheme`. The decision after reviewing this data is to replace the `nan` values and replace them with an `Unknown` value.

In [ ]:
# How to handle the missing values in the new column

data_total_merge['Pension Scheme'] = data_total_merge['Pension Scheme'].fillna('Unknown')

In [ ]:
# Checking the values in the new column

print(data_total_merge['Pension Scheme'].unique())

#### 2.3.3. Renaming features

Some features need to be renamed to help describe what teh column is for.

In [ ]:
# Renaming of Attributes

data_total_merge.rename(columns = {'Usage Code for Person : HMLR and CS General Entry Dates : CS Current Entry Date':
                                   'Civil Service Entry Date'}, inplace = True)

data_total_merge.rename(columns = {'Action Name':'Leaving Reason'}, inplace = True)

data_total_merge.rename(columns = {'Full/Part Time':'Fulltime=1/Part-time=0'}, inplace = True)

data_total_merge.rename(columns = {'Termination Date_x':'Leaving Date'}, inplace = True)

#### 2.3.4. Dropping features

There are a number of features that are not required. This is for different reasons such as:

- Not enough date: `Pension opt out date`
- Duplicated features from merging dataframes: `System Person Type_x`, `Termination Date_y`
- Required for steps after modelling: `Department`, `Cost Centre`
- Other features provide the same data: `Full-Time Equivalent`

In [ ]:
data_total_merge = data_total_merge.drop(['System Person Type_x','Employee Type',
                                          'Full-Time Equivalent','Payroll Status',
                                          'Action Reason','Effective Start Date',
                                          'Effective End Date','Input Value Name',
                                          'Pension opt out date','Termination Date_y',
                                          'System Person Type_y','Cost Centre',
                                          'Department', 'Assignment Category',
                                          'Element Name'], axis=1)

In [ ]:
univariate_stats(data_total_merge)

#### 2.3.5. Feature Engineering & Creation

There are a number of attributes that we need to calculate from the date fields such as:
- `Age (Years)`: Age of employee when leaving
- `CS LOS (Years)`: Civil Service Length of Service
- `HMLR LOS (Years)`: HM Land Registry Length of Service

To do this I have calculated the difference in the relevant months before converting this to years.

In [ ]:
# Calculate age at point of leaving in months

data_total_merge['Leaving Date'] = pd.to_datetime(data_total_merge['Leaving Date'],dayfirst=True)
data_total_merge['Date of Birth'] = pd.to_datetime(data_total_merge['Date of Birth'],dayfirst=True)

data_total_merge['Age (Months)'] = (data_total_merge['Leaving Date'] - data_total_merge['Date of Birth']).astype('<m8[M]')

# Calculate CS LOS in months

data_total_merge['Leaving Date'] = pd.to_datetime(data_total_merge['Leaving Date'],dayfirst=True)
data_total_merge['Civil Service Entry Date'] = pd.to_datetime(data_total_merge['Civil Service Entry Date'],dayfirst=True)

data_total_merge['CS LOS (Months)'] = (data_total_merge['Leaving Date'] - data_total_merge['Civil Service Entry Date']).astype('<m8[M]')

# Calculate HMLR LOS in months

data_total_merge['Leaving Date'] = pd.to_datetime(data_total_merge['Leaving Date'],dayfirst=True)
data_total_merge['Enterprise Hire Date'] = pd.to_datetime(data_total_merge['Enterprise Hire Date'],dayfirst=True)

data_total_merge['HMLR LOS (Months)'] = (data_total_merge['Leaving Date'] - data_total_merge['Enterprise Hire Date']).astype('<m8[M]')

In [ ]:
# Change date calculations from months to years

data_total_merge['Age (years)'] = round((data_total_merge['Age (Months)'] / 12),2)
data_total_merge['CS LOS (years)'] = round((data_total_merge['CS LOS (Months)'] / 12),2)
data_total_merge['HMLR LOS (years)'] = round((data_total_merge['HMLR LOS (Months)'] / 12),2)

data_total_merge.head()

#### 2.3.6. Dealing with the period of the Pandemic

To try to take into account the impact that the Pandemic had on the behaviour of staff I will create a feature to categorise the date that someone left HM Land Registry as being Pre, During or Post Pandemic.

The dates selected for the During Pandemic Period starts at the point that the first lockdown started: `01 March 2020`, until `31 March 2022` when the last restrictions were lifted.

In [ ]:
# function to assign the data row a Pre, during or Post Pandemic category.

def calculate_result(date):
    # Convert the date to a datetime object
    date = pd.to_datetime(date)

    if date < pd.to_datetime('2020-03-01'):
        return "Pre-Pandemic"
    elif pd.to_datetime('2020-03-01') <= date <= pd.to_datetime('2022-03-31'):
        return "During-Pandemic"
    else:
        return "Post-Pandemic"

In [ ]:
# Creating the Pandemic feature

data_total_merge['Pandemic'] = data_total_merge['Leaving Date'].apply(calculate_result)

data_total_merge.head()

In [ ]:
# Deleting the features that are no longer required after the creation of the new features

data_total_merge = data_total_merge.drop(['Leaving Date','Enterprise Hire Date','Date of Birth','Civil Service Entry Date',
                                          'Age (Months)','CS LOS (Months)','HMLR LOS (Months)'], axis=1)

In [ ]:
data_total_merge = data_total_merge.astype({'Person Number': 'str'})

In [ ]:
univariate_stats(data_total_merge)

### 2.4. Visualising features

#### 2.4.1. Bar plots

In [ ]:
sns.set_style('whitegrid')
sns.set_palette("tab20")

# Creating counts and columns
column1 = data_total_merge['Location']
counts1 = data_total_merge['Location'].value_counts()

column2 = data_total_merge['Grade']
counts2 = data_total_merge['Grade'].value_counts()

column3 = data_total_merge['Directorate']
counts3 = data_total_merge['Directorate'].value_counts()

column4 = data_total_merge['Pension Scheme']
counts4 = data_total_merge['Pension Scheme'].value_counts()

fig, axes = plt.subplots(4, 1, figsize=(20, 15))

# Create a bar plot for Location in the first subplot
sns.countplot(x=column1, data=data_total_merge, ax=axes[0])
axes[0].set_title('Bar plot for Location')

sns.countplot(x=column2, data=data_total_merge, ax=axes[1])
axes[1].set_title('Bar plot for Grade')

sns.countplot(x=column3, data=data_total_merge, ax=axes[2])
axes[2].set_title('Bar plot for Directorate')

sns.countplot(x=column4, data=data_total_merge, ax=axes[3])
axes[3].set_title('Bar plot for Pension Scheme')

# Adjust layout and show the figure
plt.tight_layout()
plt.show()

In [ ]:
sns.set_style('whitegrid')
sns.set_palette("tab20")

# Creating counts and columns
column5 = data_total_merge['Pandemic']
counts5 = data_total_merge['Pandemic'].value_counts()

column6 = data_total_merge['Fulltime=1/Part-time=0']
counts6 = data_total_merge['Fulltime=1/Part-time=0'].value_counts()

column7 = data_total_merge['Person Gender']
counts7 = data_total_merge['Person Gender'].value_counts()

column8 = data_total_merge['Partial_Retirement']
counts8 = data_total_merge['Partial_Retirement'].value_counts()

column9 = data_total_merge['Leaving Reason']
counts9 = data_total_merge['Leaving Reason'].value_counts()

fig, axes = plt.subplots(5, 1, figsize=(20, 15))

# Create a bar plot for Location in the first subplot
sns.countplot(x=column5, data=data_total_merge, ax=axes[0])
axes[0].set_title('Bar plot for Pandemic')

sns.countplot(x=column9, data=data_total_merge, ax=axes[1])
axes[1].set_title('Bar plot for Leaving Reason')

sns.countplot(x=column6, data=data_total_merge, ax=axes[2])
axes[2].set_title('Bar plot for Working Pattern')

sns.countplot(x=column7, data=data_total_merge, ax=axes[3])
axes[3].set_title('Bar plot for Gender')

sns.countplot(x=column8, data=data_total_merge, ax=axes[4])
axes[4].set_title('Bar plot for Partial Retirement')

# Adjust layout and show the figure
plt.tight_layout()
plt.show()

#### 2.4.2. Count plots

In [ ]:
sns.set_style('whitegrid')
sns.set_palette("tab20")

fig, axes = plt.subplots(3, 1, figsize=(20, 15))

# Create data groups for plotting
data_total_merge["Age_Group"] = pd.cut(data_total_merge["Age (years)"], bins=[0, 20, 30, 40, 50, 55, 60, 65, 70, 100], 
                                       labels=["<20", "20-29", "30-39", "40-49", "50-54", "55-59", "60-64", "65-69", "70+"])

data_total_merge["HMLR LOS"] = pd.cut(data_total_merge["HMLR LOS (years)"], bins=[0, 1, 3, 5, 10, 15, 20, 25, 30, 35, 40, 100], 
                                      labels=["<1", "1-3", "4-5", "6-10", "11-15", "16-20", "21-25", "26-30", "31-35", "36-40", ">40"])

data_total_merge["CS LOS"] = pd.cut(data_total_merge["CS LOS (years)"], bins=[0, 1, 3, 5, 10, 15, 20, 25, 30, 35, 40, 100], 
                                      labels=["<1", "1-3", "4-5", "6-10", "11-15", "16-20", "21-25", "26-30", "31-35", "36-40", ">40"])

# Using countplot for displaying counts of each group:
sns.countplot(x="Age_Group", data=data_total_merge, ax=axes[0])
axes[0].set_title("Age Group Distribution")
plt.xlabel("Age Group")
plt.ylabel("Count")
#plt.show()

sns.countplot(x="HMLR LOS", data=data_total_merge, ax=axes[1])
axes[1].set_title("HMLR Length of Service Distribution")
plt.xlabel("HMLR LOS")
plt.ylabel("Count")
#plt.show()

sns.countplot(x="CS LOS", data=data_total_merge, ax=axes[2])
axes[2].set_title("Civil Service Length of Service Distribution")
plt.xlabel("CS LOS")
plt.ylabel("Count")
#plt.show()

#sns.barplot(data=data_total_merge, x='CS LOS (years)', bins=10, ax=axes[2])
#axes[2].set_title('Civil Service - Length of Service (years)')

# Adjust layout and show the figure
plt.tight_layout()
plt.show()

Showing these visuals filtered by Leaving Reason = Retirement

In [ ]:
filtered_data = data_total_merge[data_total_merge['Leaving Reason'] == 'Retirement']

In [ ]:
sns.set_style('whitegrid')
sns.set_palette("tab20")

fig, axes = plt.subplots(3, 1, figsize=(20, 15))

# Create data groups for plotting
filtered_data["Age_Group"] = pd.cut(filtered_data["Age (years)"], bins=[0, 20, 30, 40, 50, 55, 60, 65, 70, 100], 
                                       labels=["<20", "20-29", "30-39", "40-49", "50-54", "55-59", "60-64", "65-69", "70+"])

filtered_data["HMLR LOS"] = pd.cut(filtered_data["HMLR LOS (years)"], bins=[0, 1, 3, 5, 10, 15, 20, 25, 30, 35, 40, 100], 
                                      labels=["<1", "1-3", "4-5", "6-10", "11-15", "16-20", "21-25", "26-30", "31-35", "36-40", ">40"])

filtered_data["CS LOS"] = pd.cut(filtered_data["CS LOS (years)"], bins=[0, 1, 3, 5, 10, 15, 20, 25, 30, 35, 40, 100], 
                                      labels=["<1", "1-3", "4-5", "6-10", "11-15", "16-20", "21-25", "26-30", "31-35", "36-40", ">40"])

# Using countplot for displaying counts of each group:
sns.countplot(x="Age_Group", data=filtered_data, ax=axes[0])
axes[0].set_title("Age Group Distribution - Retirements")
plt.xlabel("Age Group")
plt.ylabel("Count")
#plt.show()

sns.countplot(x="HMLR LOS", data=filtered_data, ax=axes[1])
axes[1].set_title("HMLR Length of Service Distribution - Retirements")
plt.xlabel("HMLR LOS")
plt.ylabel("Count")
#plt.show()

sns.countplot(x="CS LOS", data=filtered_data, ax=axes[2])
axes[2].set_title("Civil Service Length of Service Distribution - Retirements")
plt.xlabel("CS LOS")
plt.ylabel("Count")
#plt.show()

#sns.barplot(data=data_total_merge, x='CS LOS (years)', bins=10, ax=axes[2])
#axes[2].set_title('Civil Service - Length of Service (years)')

# Adjust layout and show the figure
plt.tight_layout()
plt.show()

In [ ]:
sns.set_style('whitegrid')
sns.set_palette("tab20")

# Creating counts and columns
column5 = filtered_data['Pandemic']
counts5 = filtered_data['Pandemic'].value_counts()

column6 = filtered_data['Fulltime=1/Part-time=0']
counts6 = filtered_data['Fulltime=1/Part-time=0'].value_counts()

column7 = filtered_data['Person Gender']
counts7 = filtered_data['Person Gender'].value_counts()

column8 = filtered_data['Partial_Retirement']
counts8 = filtered_data['Partial_Retirement'].value_counts()

column9 = filtered_data['Leaving Reason']
counts9 = filtered_data['Leaving Reason'].value_counts()

fig, axes = plt.subplots(5, 1, figsize=(20, 15))

# Create a bar plot for Location in the first subplot
sns.countplot(x=column5, data=filtered_data, ax=axes[0])
axes[0].set_title('Bar plot for Pandemic')

sns.countplot(x=column9, data=filtered_data, ax=axes[1])
axes[1].set_title('Bar plot for Leaving Reason')

sns.countplot(x=column6, data=filtered_data, ax=axes[2])
axes[2].set_title('Bar plot for Working Pattern')

sns.countplot(x=column7, data=filtered_data, ax=axes[3])
axes[3].set_title('Bar plot for Gender')

sns.countplot(x=column8, data=filtered_data, ax=axes[4])
axes[4].set_title('Bar plot for Partial Retirement')

# Adjust layout and show the figure
plt.tight_layout()
plt.show()

In [ ]:
### Need to calculate the numbers here to decide if this is imbalanced

### 2.5. Feature Relationship

#### 2.5.1. Calculation of Bivariate Stats

In [ ]:
def Anova(df, feature, label):
    import pandas as pd
    import numpy as np
    from scipy import stats
    
    groups = df[feature].unique()

    data_grouped = df.groupby(feature)

    group_labels = []
    for g in groups:
        g_list = data_grouped.get_group(g)
        group_labels.append(g_list[label])
        
    return stats.f_oneway(*group_labels)

In [ ]:
def bivstats(df, label):
    from scipy import stats
    import pandas as pd
    
    # Stat = r correlation, +/- = Anova, Effect Size = Chi Squared, p-value = p-value.
        
    corr_df = pd.DataFrame(columns=['Stat', '+/-', 'Effect size', 'p-value'])
        
    for col in df:
        if not col == label:
            if df[col].isnull().sum() == 0:
            
                if pd.api.types.is_numeric_dtype(df[col]):
                    r,p = stats.pearsonr(df[label], df[col])
                    corr_df.loc[col] = ['r', np.sign(r), abs(round(r, 3)), round(p, 6)] 
                else:
                    F, p = Anova(df[[col, label]], col, label)
                    corr_df.loc[col] = ['F', '', round(F, 3), round(p, 6)]
            else:
                corr_df.loc[col] = [np.nan, np.nan, np.nan, 'nulls']        
                     
    corr_df.sort_values(by=['Effect size'], ascending=False)
    
    return corr_df

In [ ]:
pd.options.display.float_format = '{:.5f}'.format
bivstats(data_total_merge, 'Age (years)')

#### 2.5.2. Pairplots

In [ ]:
sns.pairplot(data_total_merge, vars=['Age (years)', 'HMLR LOS (years)', 'CS LOS (years)', 'Partial_Retirement'])
    
plt.show()   

In [ ]:
sns.pairplot(data_total_merge, vars=['Age (years)', 'HMLR LOS (years)', 'CS LOS (years)','Partial_Retirement'],hue='Leaving Reason')    
plt.savefig('visualisation1.png', dpi=300, bbox_inches='tight')
plt.show() 

In [ ]:
sns.pairplot(data_total_merge, vars=['Age (years)', 'HMLR LOS (years)', 'CS LOS (years)','Partial_Retirement'],hue='Person Gender')    
plt.savefig('visualisation2.png', dpi=300, bbox_inches='tight')
plt.show() 

In [ ]:
sns.pairplot(data_total_merge, vars=['Age (years)', 'HMLR LOS (years)', 'CS LOS (years)','Partial_Retirement'],hue='Pension Scheme')    
plt.savefig('visualisation3.png', dpi=300, bbox_inches='tight')
plt.show() 

In [ ]:
sns.pairplot(data_total_merge, vars=['Age (years)', 'HMLR LOS (years)', 'CS LOS (years)','Partial_Retirement'],hue='Pandemic')    
plt.savefig('visualisation4.png', dpi=300, bbox_inches='tight')
plt.show() 

#### 2.5.3. Heatmap

In [ ]:
df_corr = filtered_data[['Age (years)','HMLR LOS (years)', 'CS LOS (years)', 'Partial_Retirement']].corr()

In [ ]:
sns.heatmap(df_corr, annot=True)

In [ ]:
### Maybe some box plots to review outliers a little more.

## 3. Modelling Preparation

This section will look at final feature adaptions to prepare for the data to then be split for training and testing of the models that have been chosen to compare.

### 3.1. Final feature changes

After visualising the features it has shown some areas that will be best to adapt further to be binary results. These are the features that do not require any more detailed processing such as One Hot Encoding. 

#### Person Gender

In [ ]:
print(data_total_merge['Person Gender'].unique())

In [ ]:
data_total_merge.loc[(data_total_merge['Person Gender'] == 'Male'), 'Person Gender'] = 1 
data_total_merge.loc[(data_total_merge['Person Gender'] == 'Female'), 'Person Gender'] = 0

#### Fulltime / Part-time

In [ ]:
print(data_total_merge['Fulltime=1/Part-time=0'].unique())

In [ ]:
data_total_merge.loc[(data_total_merge['Fulltime=1/Part-time=0'] == 'Full time'), 'Fulltime=1/Part-time=0'] = 1 
data_total_merge.loc[(data_total_merge['Fulltime=1/Part-time=0'] == 'Part time'), 'Fulltime=1/Part-time=0'] = 0

#### Leaving Reason

The important question for this column is whether the 'Leaving Reason' is Retirement or not. Therefore I will amend the values to be a numerical binary view, Retirement = 1 and Non-Retirement = 0.

In [ ]:
print(data_total_merge['Leaving Reason'].unique())

In [ ]:
data_total_merge['Outcome'] = np.where(data_total_merge['Leaving Reason'] != 'Retirement', False, True)

In [ ]:
data_total_merge.head()

In [ ]:
data_total_merge.to_csv('final merge test.csv', index=False)

### 3.2. One Hot Encoding

In [ ]:
one_hot_encoded_data = pd.get_dummies(data_total_merge, columns = ['Location','Grade','Directorate',
                                                                   'Pension Scheme','Pandemic'])
one_hot_encoded_data.head()

In [ ]:
# Testing that the code has worked as I expected.
one_hot_encoded_data.to_csv('Test Review Data.csv', index=False)

In [ ]:
full_corr = one_hot_encoded_data.corr()

plt.figure(figsize=(30,30))
sns.heatmap(full_corr, annot=False)
plt.savefig('visualisation5.png', dpi=300, bbox_inches='tight')
plt.show() 

### 3.3. Normalisation

In [ ]:
scaler = MinMaxScaler()
cols_to_norm = ['Age (years)', 'CS LOS (years)','HMLR LOS (years)']
one_hot_encoded_data[cols_to_norm] = scaler.fit_transform(one_hot_encoded_data[cols_to_norm])

In [ ]:
# Testing that the code has worked as I expected.

one_hot_encoded_data.to_csv('Test Review Data2.csv', index=False)

### 3.4. Training - Testing split

In [ ]:
#Assigning the dataset to input & output

x = one_hot_encoded_data[['Person Number','Fulltime=1/Part-time=0','Person Gender', 
                               'Partial_Retirement','Age (years)','CS LOS (years)','HMLR LOS (years)',
                               'Location_Birkenhead','Location_Coventry', 'Location_Croydon', 'Location_Durham', 
                               'Location_Fylde','Location_Gloucester', 'Location_Hull', 'Location_Leicester', 
                               'Location_Nottingham','Location_Peterborough','Location_Plymouth','Location_Swansea', 
                               'Location_Telford','Location_Weymouth','Grade_AA','Grade_AO','Grade_APP','Grade_EO',
                               'Grade_Grade 6','Grade_Grade 7','Grade_Grade 7 Lawyers','Grade_HEO','Grade_SCS1',
                               'Grade_SCS2','Grade_SEO', 'Grade_SEO+', 'Directorate_CE & CLR', 
                               'Directorate_Customer & Strategy Group','Directorate_DDat',
                               'Directorate_Data & Register Integrity Group', 'Directorate_FBS',
                               'Directorate_Human Resources', 'Directorate_Independent Complaints Review', 
                               'Directorate_Operations', 'Directorate_Transformation', 'Pension Scheme_Alpha', 
                               'Pension Scheme_Classic', 'Pension Scheme_Classic Plus', 'Pension Scheme_L&G', 
                               'Pension Scheme_Nuvos', 'Pension Scheme_Premium', 'Pension Scheme_Unknown', 
                               'Pandemic_During-Pandemic', 'Pandemic_Post-Pandemic', 'Pandemic_Pre-Pandemic']]
y = one_hot_encoded_data[['Outcome']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size = 0.2)

In [ ]:
X_train.to_csv('X_train.csv',index=False)
X_test.to_csv('X_test.csv', index=False)
y_train.to_csv('y_train.csv',index=False)
y_test.to_csv('y_test.csv', index=False)

## 4. Classification Models

This problem is a classification models and due to this and the research that I have completed I will be using the following algorithms to optimise, train & test:

1. Random Forest (rf)
2. Support Vector Classification (svc) / Support Vector Machines (svm)
3. Logistic Regression (lg)
4. K-Nearest Neighbours (knn)

For each of these algorithms I will optimise the hyperparameters, complete a single run and then complete a multi run and calculate averages over a number of metrics.

### 4.1. Random Forest

#### 4.1.1. Random Forest - First Run

I will start by building the model and running this with the default parameters.

In [ ]:
rf_model = RandomForestClassifier()

In [ ]:
rf_model.fit(X_train, y_train)

In [ ]:
rf_y_predicted = rf_model.predict(X_test)

In [ ]:
rf_y_predicted = rf_model.predict(X_test)

rf_cm = confusion_matrix(y_test, rf_y_predicted)
rf_cm

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(rf_cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel('Predicted')
plt.ylabel('Truth')
plt.show()

In [ ]:
print(accuracy_score(y_test, rf_y_predicted))

In [ ]:
print('F1 Score: ', f1_score(y_test, rf_y_predicted))

In [ ]:
rfc_fpr_rf, rfc_tpr_rf, threshold_rf = roc_curve(y_test, rf_y_predicted)
rf_auc_rfc = auc(rfc_fpr_rf, rfc_tpr_rf)

print("AUC: ",rf_auc_rfc)

In [ ]:
plt.figure(figsize=(5,5), dpi=100)
plt.plot(rfc_fpr_rf, rfc_tpr_rf, linestyle='-', label='Random Forest (AUC = %0.3f)' % rf_auc_rfc)

plt.xlabel('False Positive Rate -->')
plt.ylabel('True Positive rate -->')

plt.legend()

plt.show()

#### 4.1.2. Random Forest - Tuning Hyperparameters

In [ ]:
rf_start_time_training = time.time()

param_grid_rf = [{
    'n_estimators':[10,20,30,40,50,60,70,80,90,100,110,120,130,140,150],
    'max_depth':[5,10,15,20],
    'max_features':[2,3,4,5,6,7,8,9,10]},]

rf_optimal_params = GridSearchCV(estimator=rf_model, param_grid=param_grid_rf, cv=10, n_jobs=-1)

rf_best_model = rf_optimal_params.fit(X_train, y_train)

rf_end_time_training = time.time()

print(rf_optimal_params.best_params_)

In [ ]:
rf_opt_y_predicted = rf_best_model.predict(X_test)

In [ ]:
rf_cm_opt = confusion_matrix(y_test, rf_opt_y_predicted)
rf_cm_opt

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(rf_cm_opt, annot=True, fmt="d", cmap="Blues")
plt.xlabel('Predicted')
plt.ylabel('Truth')
plt.show()

In [ ]:
print(accuracy_score(y_test, rf_opt_y_predicted))

In [ ]:
print('F1 Score: ', f1_score(y_test, rf_opt_y_predicted))

In [ ]:
rfc_fpr_rf_opt, rfc_tpr_rf_opt, threshold_rf_opt = roc_curve(y_test, rf_opt_y_predicted)
rf_opt_auc_rfc = auc(rfc_fpr_rf_opt, rfc_tpr_rf_opt)

print("AUC: ",rf_opt_auc_rfc)

In [ ]:
plt.figure(figsize=(5,5), dpi=100)
plt.plot(rfc_fpr_rf_opt, rfc_tpr_rf_opt, linestyle='-', label='Random Forest (AUC = %0.3f)' % rf_opt_auc_rfc)

plt.xlabel('False Positive Rate -->')
plt.ylabel('True Positive rate -->')

plt.legend()

plt.show()

In [ ]:
rf_training_time = rf_end_time_training - rf_start_time_training
print("Training time:", rf_training_time, "seconds")
print("")

In [ ]:
rf_model_multi = RandomForestClassifier(max_depth=5, max_features=9, n_estimators=30)
rf_model_multi.fit(X_train, y_train)

In [ ]:
rf_start_time_testing = time.time()

rf_accuracy=[]
rf_f1score=[]
rf_roc=[]

for i in range(100):    
    rf_y_predicted_multi = rf_best_model.predict(X_test)
    
    acc = accuracy_score(y_test, rf_y_predicted_multi)
    rf_accuracy.append(acc)
    
    f1 = f1_score(y_test, rf_y_predicted_multi)
    rf_f1score.append(f1)
    
    auc = roc_auc_score(y_test, rf_y_predicted_multi)
    rf_roc.append(auc)
    
rf_end_time_testing = time.time()

In [ ]:
rf_training_time = rf_end_time_training - rf_start_time_training
print("Training time:", rf_training_time, "seconds")
print("")

rf_testing_time = rf_end_time_testing - rf_start_time_testing
print("Testing time:", rf_testing_time, "seconds")
print("")

acc_average = statistics.mean(rf_accuracy)
f1_average = statistics.mean(rf_f1score)
roc_average = statistics.mean(rf_roc)

print('This is the accuracy average', acc_average)
print('This is the F1 average', f1_average)
print('This is the AUC average', roc_average)
print("")

acc_max = max(rf_accuracy)
f1_max = max(rf_f1score)
roc_max = max(rf_roc)

print('This is the accuracy maximum', acc_max)
print('This is the F1 maximum', f1_max)
print('This is the AUC maximum', roc_max)
print("")

acc_min = min(rf_accuracy)
f1_min = min(rf_f1score)
roc_min = min(rf_roc)

print('This is the accuracy minimum', acc_min)
print('This is the F1 minimum', f1_min)
print('This is the AUC minimum', roc_min)
print("")

acc_stdev = np.std(rf_accuracy)
f1_stdev = np.std(rf_f1score)
roc_stdev = np.std(rf_roc)

print('This is the accuracy standard deviation', acc_stdev)
print('This is the F1 standard deviation', f1_stdev)
print('This is the AUC standard deviation', roc_stdev)

### 4.2. Support Vector Machines (SVM)

#### 4.2.1. SVM - First Run

In [ ]:
SVM_model = SVC()
SVM_model.fit(X_train, y_train)

In [ ]:
SVM_y_predicted = SVM_model.predict(X_test)

In [ ]:
SVM_cm = confusion_matrix(y_test, SVM_y_predicted)
SVM_cm

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(SVM_cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel('Predicted')
plt.ylabel('Truth')
plt.show()

In [ ]:
print(accuracy_score(y_test, SVM_y_predicted))

In [ ]:
print('F1 Score: ', f1_score(y_test, SVM_y_predicted))

In [ ]:
rfc_fpr_SVM, rfc_tpr_SVM, threshold_SVM = roc_curve(y_test, SVM_y_predicted)
auc_rfc_SVM = auc(rfc_fpr_SVM, rfc_tpr_SVM)

print('auc_rfc_SVM type:', type(auc_rfc_SVM))
print("AUC: ", auc_rfc_SVM)

In [ ]:
plt.figure(figsize=(5,5), dpi=100)
plt.plot(rfc_fpr_SVM, rfc_tpr_SVM, linestyle='-', label='SVM (AUC = %0.3f)' % auc_rfc_SVM)

plt.xlabel('False Positive Rate -->')
plt.ylabel('True Positive rate -->')

plt.legend()

plt.show()

#### 4.2.2. SVM - Tuning hyperparameters

In [ ]:
SVM_start_time_training = time.time()

param_grid_SVM = [{
    'C':[0.5, 1, 10, 100],
    'gamma':[1, 0.1, 0.01, 0.001, 0.0001],
    'kernel':['rbf','linear']},]

SVM_optimal_params = GridSearchCV(estimator=SVM_model, param_grid=param_grid_SVM, cv=10, n_jobs=-1)

SVM_best_model = SVM_optimal_params.fit(X_train, y_train)

SVM_end_time_training = time.time()

print(SVM_optimal_params.best_params_)

In [ ]:
SVM_opt_y_predicted = SVM_best_model.predict(X_test)

In [ ]:
SVM_opt_cm = confusion_matrix(y_test, SVM_opt_y_predicted)
SVM_opt_cm

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(SVM_opt_cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel('Predicted')
plt.ylabel('Truth')
plt.show()

In [ ]:
print(accuracy_score(y_test, SVM_opt_y_predicted))

In [ ]:
print('F1 Score: ', f1_score(y_test, SVM_opt_y_predicted))

In [ ]:
rfc_fpr_SVM_opt, rfc_tpr_SVM_opt, threshold_SVM_opt = roc_curve(y_test, SVM_opt_y_predicted)
auc_rfc_SVM_opt = auc(rfc_fpr_SVM_opt, rfc_tpr_SVM_opt)

print('AUC: ', auc_rfc_SVM_opt)

In [ ]:
plt.figure(figsize=(5,5), dpi=100)
plt.plot(rfc_fpr_SVM_opt, rfc_tpr_SVM_opt, linestyle='-', label='SVM - Optimised (AUC = %0.3f)' % auc_rfc_SVM_opt)

plt.xlabel('False Positive Rate -->')
plt.ylabel('True Positive rate -->')

plt.legend()

plt.show()

In [ ]:
SVM_training_time = SVM_end_time_training - SVM_start_time_training
print("Training time:", SVM_training_time, "seconds")
print("")

In [ ]:
SVM_model_multi = SVC(C=1, gamma=0.0001, kernel='rbf')
SVM_model_multi.fit(X_train, y_train)

In [ ]:
SVM_start_time_testing = time.time()

SVM_accuracy=[]
SVM_f1score=[]
SVM_roc=[]

for i in range(100):
    SVM_y_predicted_multi = SVM_best_model.predict(X_test)
    
    acc = accuracy_score(y_test, SVM_y_predicted_multi)
    SVM_accuracy.append(acc)
    
    f1 = f1_score(y_test, SVM_y_predicted_multi)
    SVM_f1score.append(f1)
    
    auc = roc_auc_score(y_test, SVM_y_predicted_multi)
    SVM_roc.append(auc)
    
SVM_end_time_testing = time.time()

In [ ]:
SVM_training_time = SVM_end_time_training - SVM_start_time_training
print("Training time:", SVM_training_time, "seconds")
print("")

SVM_testing_time = SVM_end_time_testing - SVM_start_time_testing
print("Testing time:", SVM_testing_time, "seconds")
print("")

acc_average = statistics.mean(SVM_accuracy)
f1_average = statistics.mean(SVM_f1score)
roc_average = statistics.mean(SVM_roc)

print('This is the accuracy average', acc_average)
print('This is the F1 average', f1_average)
print('This is the AUC average', roc_average)
print("")

acc_max = max(SVM_accuracy)
f1_max = max(SVM_f1score)
roc_max = max(SVM_roc)

print('This is the accuracy maximum', acc_max)
print('This is the F1 maximum', f1_max)
print('This is the AUC maximum', roc_max)
print("")

acc_min = min(SVM_accuracy)
f1_min = min(SVM_f1score)
roc_min = min(SVM_roc)

print('This is the accuracy minimum', acc_min)
print('This is the F1 minimum', f1_min)
print('This is the AUC minimum', roc_min)
print("")

acc_stdev = np.std(SVM_accuracy)
f1_stdev = np.std(SVM_f1score)
roc_stdev = np.std(SVM_roc)

print('This is the accuracy standard deviation', acc_stdev)
print('This is the F1 standard deviation', f1_stdev)
print('This is the AUC standard deviation', roc_stdev)

### 4.3. Logistic Regression

#### 4.3.1. Logistic Regression - First Run

In [ ]:
lg_model = LogisticRegression()
lg_model.fit(X_test, y_test)

In [ ]:
lg_y_predicted = lg_model.predict(X_test)

In [ ]:
lg_cm = confusion_matrix(y_test, lg_y_predicted)
lg_cm

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(lg_cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel('Predicted')
plt.ylabel('Truth')
plt.show()

In [ ]:
print('Accuracy: ', accuracy_score(y_test, lg_y_predicted))

In [ ]:
print('F1 Score: ', f1_score(y_test, lg_y_predicted))

In [ ]:
rfc_fpr_lg, rfc_tpr_lg, threshold_lg = roc_curve(y_test, lg_y_predicted)
auc_rfc_lg = auc(rfc_fpr_lg, rfc_tpr_lg)

print('AUC: ', auc_rfc_lg)

In [ ]:
plt.figure(figsize=(5,5), dpi=100)
plt.plot(rfc_fpr_lg, rfc_tpr_lg, linestyle='-', label='Logistic Regression (AUC = %0.3f)' % auc_rfc_lg)

plt.xlabel('False Positive Rate -->')
plt.ylabel('True Positive rate -->')

plt.legend()

plt.show()

#### 4.3.2. Logistic Regression - Tuning hyperparameters

In [ ]:
lg_start_time_training = time.time()

param_grid_lg = [{
    'C': np.logspace(-4,4,20),
    'solver': ['lbfgs','newton-cg','liblinear','sag','saga'],
    'max_iter': [100,1000,2500,5000]},]


lg_optimal_params = GridSearchCV(estimator=lg_model, param_grid= param_grid_lg, cv=10, n_jobs=-1)

lg_best_model = lg_optimal_params.fit(X_train, y_train)

lg_end_time_training = time.time()

print(lg_optimal_params.best_params_)

In [ ]:
lg_opt_y_predicted = lg_best_model.predict(X_test)

In [ ]:
lg_opt_cm = confusion_matrix(y_test, lg_opt_y_predicted)
lg_opt_cm

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(lg_opt_cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel('Predicted')
plt.ylabel('Truth')
plt.show()

In [ ]:
print('Accuracy: ', accuracy_score(y_test, lg_opt_y_predicted))

In [ ]:
print('F1 Score: ', f1_score(y_test, lg_opt_y_predicted))

In [ ]:
rfc_fpr_lg_opt, rfc_tpr_lg_opt, threshold_lg_opt = roc_curve(y_test, lg_opt_y_predicted)
auc_rfc_lg_opt = auc(rfc_fpr_lg_opt, rfc_tpr_lg_opt)

print('AUC: ', auc_rfc_lg_opt)

In [ ]:
plt.figure(figsize=(5,5), dpi=100)
plt.plot(rfc_fpr_lg_opt, rfc_tpr_lg_opt, linestyle='-', label='Logistic Regression (AUC = %0.3f)' % auc_rfc_lg_opt)

plt.xlabel('False Positive Rate -->')
plt.ylabel('True Positive rate -->')

plt.legend()

plt.show()

In [ ]:
lg_training_time = lg_end_time_training - lg_start_time_training
print("Training time:", lg_training_time, "seconds")
print("")

In [ ]:
#May not need the cell below!!!

In [ ]:
lg_start_time_training_1 = time.time()

lg_accuracy = []
lg_f1score = []
lg_roc = []

for i in range(100):
    lg_multi = LogisticRegression()
    
    param_grid_lg = [{
    'C': np.logspace(-4,4,20),
    'solver': ['lbfgs','newton-cg','liblinear','sag','saga'],
    'max_iter': [100,1000,2500,5000]},]
    
    lg_optimal_params = GridSearchCV(estimator=lg_multi, param_grid= param_grid_lg, cv=10, n_jobs=-1)
    lg_best_model = lg_optimal_params.fit(X_train, y_train)
    
    lg_y_predicted_multi = lg_best_model.predict(X_test)
    
    acc = accuracy_score(y_test, lg_y_predicted_multi)
    lg_accuracy.append(acc)
    
    f1 = f1_score(y_test, lg_y_predicted_multi)
    lg_f1score.append(f1)
    
    auc = roc_auc_score(y_test, lg_y_predicted_multi)
    lg_roc.append(auc)
    
lg_end_time_training_1 = time.time()

print(lg_optimal_params.best_params_)

In [ ]:
#'C': 78.47599703514607, 'max_iter': 100, 'solver': 'newton-cg'

In [ ]:
lg_start_time_training = time.time()

lg_model_multi = LogisticRegression(C= 78.47599703514607, max_iter= 100, solver= 'newton-cg')
lg_model_multi.fit(X_test, y_test)

lg_end_time_training = time.time()

In [ ]:
lg_start_time_testing = time.time()

lg_accuracy=[]
lg_f1score=[]
lg_roc=[]

for i in range(100):    
    lg_y_predicted_multi = lg_best_model.predict(X_test)
    
    acc = accuracy_score(y_test, lg_y_predicted_multi)
    lg_accuracy.append(acc)
    
    f1 = f1_score(y_test, lg_y_predicted_multi)
    lg_f1score.append(f1)
    
    rfc_fpr_lg, rfc_tpr_lg, threshold_lg = roc_curve(y_test, lg_y_predicted_multi)
    auc_rfc_lg = auc(rfc_fpr_lg, rfc_tpr_lg)
    lg_roc.append(auc_rfc_lg)
    
lg_end_time_testing = time.time()

In [ ]:
lg_training_time = lg_end_time_training - lg_start_time_training
print("Training time:", lg_training_time, "seconds")
print("")

lg_testing_time = lg_end_time_testing - lg_start_time_testing
print("Testing time:", lg_testing_time, "seconds")
print("")

acc_average = statistics.mean(lg_accuracy)
f1_average = statistics.mean(lg_f1score)
roc_average = statistics.mean(lg_roc)

print('This is the accuracy average', acc_average)
print('This is the F1 average', f1_average)
print('This is the AUC average', roc_average)
print("")

acc_max = max(lg_accuracy)
f1_max = max(lg_f1score)
roc_max = max(lg_roc)

print('This is the accuracy maximum', acc_max)
print('This is the F1 maximum', f1_max)
print('This is the AUC maximum', roc_max)
print("")

acc_min = min(lg_accuracy)
f1_min = min(lg_f1score)
roc_min = min(lg_roc)

print('This is the accuracy minimum', acc_min)
print('This is the F1 minimum', f1_min)
print('This is the AUC minimum', roc_min)
print("")

acc_stdev = np.std(lg_accuracy)
f1_stdev = np.std(lg_f1score)
roc_stdev = np.std(lg_roc)

print('This is the accuracy standard deviation', acc_stdev)
print('This is the F1 standard deviation', f1_stdev)
print('This is the AUC standard deviation', roc_stdev)

### 4.4. K-Nearest Neighbour (KNN)

#### 4.4.1. KNN - First Run

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

In [ ]:
knn_y_predicted = knn.predict(X_test)

In [ ]:
knn_cm = confusion_matrix(y_test, knn_y_predicted)
knn_cm

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(knn_cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel('Predicted')
plt.ylabel('Truth')
plt.show()

In [ ]:
print(accuracy_score(y_test, knn_y_predicted))

In [ ]:
print('F1 Score: ', f1_score(y_test, knn_y_predicted))

In [ ]:
rfc_fpr_knn, rfc_tpr_knn, threshold_knn = roc_curve(y_test, knn_y_predicted)
auc_rfc_knn = auc(rfc_fpr_knn, rfc_tpr_knn)

print(auc_rfc_knn)

In [ ]:
plt.figure(figsize=(5,5), dpi=100)
plt.plot(rfc_fpr_knn, rfc_tpr_knn, linestyle='-', label='K-NN (AUC = %0.3f)' % auc_rfc_knn)

plt.xlabel('False Positive Rate -->')
plt.ylabel('True Positive rate -->')

plt.legend()

plt.show()

#### 4.4.2. KNN - Tuning hyperparameters

In [ ]:
# Optimising the hyper parameters

knn_start_time_training = time.time()

param_grid_knn = [{
        'n_neighbors': [1,2,3,4,5,6,7,8,9,10,20,30,40,50],
            },]


knn_optimal_params = GridSearchCV(estimator=knn, param_grid= param_grid_knn, cv=10, n_jobs=-1)

knn_best_model = knn_optimal_params.fit(X_train, y_train)

knn_end_time_training = time.time()

print(knn_optimal_params.best_params_)

In [ ]:
knn_opt_y_predicted = knn.predict(X_test)

In [ ]:
knn_opt_cm = confusion_matrix(y_test, knn_opt_y_predicted)
knn_opt_cm

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(knn_opt_cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel('Predicted')
plt.ylabel('Truth')
plt.show()

In [ ]:
print(accuracy_score(y_test, knn_opt_y_predicted))

In [ ]:
print('F1 Score: ', f1_score(y_test, knn_opt_y_predicted))

In [ ]:
rfc_fpr_knn_opt, rfc_tpr_knn_opt, threshold_knn_opt = roc_curve(y_test, knn_opt_y_predicted)
auc_rfc_knn_opt = auc(rfc_fpr_knn_opt, rfc_tpr_knn_opt)

print(auc_rfc_knn_opt)

In [ ]:
plt.figure(figsize=(5,5), dpi=100)
plt.plot(rfc_fpr_knn_opt, rfc_tpr_knn_opt, linestyle='-', label='K-NN (AUC = %0.3f)' % auc_rfc_knn_opt)

plt.xlabel('False Positive Rate -->')
plt.ylabel('True Positive rate -->')

plt.legend()

plt.show()

In [ ]:
knn_training_time = knn_end_time_training - knn_start_time_training
print("Training time:", knn_training_time, "seconds")
print("")

In [ ]:
knn_start_time_training_1 = time.time()

knn_accuracy = []
knn_f1score = []
knn_roc = []

for i in range(100):
    knn_multi = KNeighborsClassifier()
    param_grid_knn = [{
        'n_neighbors': [1,2,3,4,5,6,7,8,9,10,20,30,40,50],
            },]
    knn_optimal_params = GridSearchCV(estimator=knn_multi, param_grid= param_grid_knn, cv=10, n_jobs=-1)
    knn_best_model = knn_optimal_params.fit(X_train, y_train)
    knn_y_predicted_multi = knn_best_model.predict(X_test)
    
    acc = accuracy_score(y_test, knn_y_predicted_multi)
    knn_accuracy.append(acc)
    
    f1 = f1_score(y_test, knn_y_predicted_multi)
    knn_f1score.append(f1)
    
    auc = roc_auc_score(y_test, knn_y_predicted_multi)
    knn_roc.append(auc)
    
knn_end_time_training_1 = time.time()

print(knn_optimal_params.best_params_)

In [ ]:
knn_start_time_training = time.time()

knn_multi = KNeighborsClassifier(n_neighbors=50)
knn_multi.fit(X_train, y_train)

knn_end_time_training = time.time()

In [ ]:
knn_start_time_testing = time.time()

knn_accuracy=[]
knn_f1score=[]
knn_roc=[]


for i in range(100):
    knn_y_predicted_multi = knn_best_model.predict(X_test)
   
    acc = accuracy_score(y_test, knn_y_predicted_multi)
    knn_accuracy.append(acc)
   
    f1 = f1_score(y_test, knn_y_predicted_multi)
    knn_f1score.append(f1)
    
    auc = roc_auc_score(y_test, knn_y_predicted_multi)
    knn_roc.append(auc)
    
knn_end_time_testing = time.time()

In [ ]:
knn_training_time_1 = knn_end_time_training_1 - knn_start_time_training_1
print("Training time:", knn_training_time_1, "seconds")
print("")

knn_testing_time = knn_end_time_testing - knn_start_time_testing
print("Testing time:", knn_testing_time, "seconds")
print("")

acc_average = statistics.mean(knn_accuracy)
f1_average = statistics.mean(knn_f1score)
roc_average = statistics.mean(knn_roc)

print('This is the accuracy average', acc_average)
print('This is the F1 average', f1_average)
print('This is the AUC average', roc_average)
print("")

acc_max = max(knn_accuracy)
f1_max = max(knn_f1score)
roc_max = max(knn_roc)

print('This is the accuracy maximum', acc_max)
print('This is the F1 maximum', f1_max)
print('This is the AUC maximum', roc_max)
print("")

acc_min = min(knn_accuracy)
f1_min = min(knn_f1score)
roc_min = min(knn_roc)

print('This is the accuracy minimum', acc_min)
print('This is the F1 minimum', f1_min)
print('This is the AUC minimum', roc_min)
print("")

acc_stdev = np.std(knn_accuracy)
f1_stdev = np.std(knn_f1score)
roc_stdev = np.std(knn_roc)

print('This is the accuracy standard deviation', acc_stdev)
print('This is the F1 standard deviation', f1_stdev)
print('This is the AUC standard deviation', roc_stdev)

# NOTES: Try to run just the predict section through the iterative process, rather than the training and testing steps

In [ ]:
## At end of section 2
# maybe some more deep statistical testing
# talk about normality, heterostacity


# 5. Reviewing Outputs to select the model to move forward with.
# 5.1 Code to present results
# 5.2. Select the model
